# 🔄 Workflow vs Agente - Comparación 

**Módulo 1: Fundamentos**  
**Tiempo estimado**: 15 minutos  
**Objetivo**: Demostrar la diferencia fundamental entre workflows y agentes

---

## 🎯 Objetivos de Aprendizaje

Al final de esta demo, la audiencia entenderá:

1. ✅ Diferencia entre workflow (determinístico) y agente (autónomo)
2. ✅ Cuándo usar cada uno
3. ✅ Trade-offs: velocidad vs flexibilidad
4. ✅ Conceptos: `add_edge` vs `add_conditional_edges`

---

## 💬 SCRIPT DE INTRODUCCIÓN

> "Buenos días. Empecemos con la pregunta más importante cuando construyes sistemas con LLMs:
> 
> **¿Debo usar un workflow o un agente?**
> 
> En este notebook vamos a ver ambos lado a lado.
> Al final, sabrán exactamente cuándo usar cada uno."

In [1]:
# ╔════════════════════════════════════════════════════════════╗
# ║  📋 CELL 1: SETUP Y VERIFICACIÓN                          ║
# ║  ⏱️ Tiempo: 1 minuto                                       ║
# ╚════════════════════════════════════════════════════════════╝

# Instalar dependencias (solo primera vez)
%pip install -q langgraph langchain-openai python-dotenv

import os
import sys
from dotenv import load_dotenv

# Configurar path para imports
sys.path.append(os.path.abspath('../..'))

load_dotenv()

# Verificación visual
print("╔════════════════════════════════════════╗")
print("║   SETUP VERIFICATION                   ║")
print("╚════════════════════════════════════════╝")
print(f"✅ Python version: {sys.version.split()[0]}")
print(f"{'✅' if os.getenv('OPENAI_API_KEY') else '❌'} OpenAI API Key configured")
print(f"\n🎬 Ready to start!\n")

Note: you may need to restart the kernel to use updated packages.
╔════════════════════════════════════════╗
║   SETUP VERIFICATION                   ║
╚════════════════════════════════════════╝
✅ Python version: 3.13.7
✅ OpenAI API Key configured

🎬 Ready to start!



## 💡 Concepto Clave: El Espectro de Autonomía

```
WORKFLOW                                           AGENTE
Determinístico ←──────────────────────────────→ Autónomo
0% Incertidumbre                    100% Incertidumbre

Ejemplos:
│                                                    │
Pipeline ETL                          Customer Support Bot
├─── clean                            ├─── classify intent
├─── transform                        ├─── choose action
└─── load                             ├─── use tools
                                      └─── synthesize
```

### 💬 SCRIPT:

> "Piensen en esto como un espectro.
> 
> - **Izquierda** (Workflow): Sabes EXACTAMENTE qué va a pasar. Como una receta.
> - **Derecha** (Agente): El LLM DECIDE qué hacer. Como un chef improvisando.
> 
> La mayoría de sistemas reales están en el MEDIO - combinan ambos."

In [2]:
# ╔════════════════════════════════════════════════════════════╗
# ║  📝 CELL 2: EJEMPLO 1 - WORKFLOW (DETERMINÍSTICO)         ║
# ║  ⏱️ Tiempo: 5 minutos                                      ║
# ╚════════════════════════════════════════════════════════════╝

from typing import TypedDict
from langgraph.graph import StateGraph, START, END

# Estado simple
class WorkflowState(TypedDict):
    input: str
    step1_output: str
    step2_output: str
    step3_output: str

# Nodos - funciones simples
def step1(state: WorkflowState) -> WorkflowState:
    """Paso 1: Siempre se ejecuta primero"""
    result = f"Step1({state['input']})"
    print(f"  ✓ Step 1: {result}")
    return {"step1_output": result}

def step2(state: WorkflowState) -> WorkflowState:
    """Paso 2: Siempre se ejecuta segundo"""
    result = f"Step2({state['step1_output']})"
    print(f"  ✓ Step 2: {result}")
    return {"step2_output": result}

def step3(state: WorkflowState) -> WorkflowState:
    """Paso 3: Siempre se ejecuta tercero"""
    result = f"Step3({state['step2_output']})"
    print(f"  ✓ Step 3: {result}")
    return {"step3_output": result}

# Construir workflow
workflow_graph = StateGraph(WorkflowState)
workflow_graph.add_node("step1", step1)
workflow_graph.add_node("step2", step2)
workflow_graph.add_node("step3", step3)

# 🔑 CLAVE: add_edge (NO conditional)
workflow_graph.add_edge(START, "step1")
workflow_graph.add_edge("step1", "step2")  # ← SIEMPRE este orden
workflow_graph.add_edge("step2", "step3")
workflow_graph.add_edge("step3", END)

workflow_app = workflow_graph.compile()

print("✅ Workflow compilado")
print("\n💡 Fíjense: usamos 'add_edge' - el flujo es FIJO\n")

✅ Workflow compilado

💡 Fíjense: usamos 'add_edge' - el flujo es FIJO



In [3]:
# 🎬 DEMO: Ejecutar Workflow

print("="*60)
print("🔄 EJECUTANDO WORKFLOW")
print("="*60)

result = workflow_app.invoke({"input": "test_data"})

print("\n📊 RESULTADO:")
print(f"  Input:  {result['input']}")
print(f"  Step 1: {result['step1_output']}")
print(f"  Step 2: {result['step2_output']}")
print(f"  Step 3: {result['step3_output']}")
print("\n" + "="*60)

# 💬 SCRIPT:
print("\n💬 EXPLICAR A LA AUDIENCIA:")
print("   'Fíjense: SIEMPRE los mismos 3 pasos, en el mismo orden.'")
print("   'No importa el input - el flujo es PREDECIBLE.'")

🔄 EJECUTANDO WORKFLOW
  ✓ Step 1: Step1(test_data)
  ✓ Step 2: Step2(Step1(test_data))
  ✓ Step 3: Step3(Step2(Step1(test_data)))

📊 RESULTADO:
  Input:  test_data
  Step 1: Step1(test_data)
  Step 2: Step2(Step1(test_data))
  Step 3: Step3(Step2(Step1(test_data)))


💬 EXPLICAR A LA AUDIENCIA:
   'Fíjense: SIEMPRE los mismos 3 pasos, en el mismo orden.'
   'No importa el input - el flujo es PREDECIBLE.'


In [12]:
# ╔════════════════════════════════════════════════════════════╗
# ║  📝 CELL 3: EJEMPLO 2 - AGENTE (AUTÓNOMO)                 ║
# ║  ⏱️ Tiempo: 6 minutos                                      ║
# ╚════════════════════════════════════════════════════════════╝

from typing import Annotated
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode
import operator

# Estado del agente
class AgentState(TypedDict):
    messages: Annotated[list, operator.add]

# Herramientas
@tool
def search_tool(query: str) -> str:
    """Busca información. Usa esto para preguntas que requieren búsqueda."""
    return f"Resultados de búsqueda para: {query}"

@tool
def calculator_tool(expression: str) -> str:
    """Realiza cálculos. Usa esto para operaciones matemáticas."""
    try:
        return f"Resultado: {eval(expression)}"
    except:
        return "Error en el cálculo"

# LLM con herramientas
tools = [search_tool, calculator_tool]
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
llm_with_tools = llm.bind_tools(tools)

# Nodo del agente
def agent_node(state: AgentState):
    """El agente DECIDE qué hacer"""
    response = llm_with_tools.invoke(state["messages"])
    return {"messages": [response]}

# 🔑 CLAVE: Decisión condicional
def should_continue(state: AgentState):
    """Decide si usar herramientas o terminar"""
    last_message = state["messages"][-1]
    
    # ¿El LLM quiere usar herramientas?
    if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
        return "tools"
    return "end"

# Construir agente
agent_graph = StateGraph(AgentState)
agent_graph.add_node("agent", agent_node)
agent_graph.add_node("tools", ToolNode(tools))

# 🔑 CLAVE: add_conditional_edges (decisión dinámica)
agent_graph.add_edge(START, "agent")
agent_graph.add_conditional_edges(
    "agent",
    should_continue,  # ← FUNCIÓN que decide
    {
        "tools": "tools",
        "end": END
    }
)
agent_graph.add_edge("tools", "agent")  # Loop de vuelta

agent_app = agent_graph.compile()

print("✅ Agente compilado")
print("\n💡 Fíjense: usamos 'add_conditional_edges' - el flujo es DINÁMICO\n")

✅ Agente compilado

💡 Fíjense: usamos 'add_conditional_edges' - el flujo es DINÁMICO



In [13]:
# 🎬 DEMO: Ejecutar Agente con diferentes inputs

test_cases = [
    "¿Cuánto es 5 + 3?",  # Necesita calculator
    "¿Qué es LangGraph?",  # Necesita search
    "Hola, ¿cómo estás?"  # No necesita herramientas
]

for i, test_input in enumerate(test_cases, 1):
    print("="*60)
    print(f"🤖 CASO {i}: Agente con input variado")
    print("="*60)
    print(f"📝 Input: {test_input}\n")
    
    result = agent_app.invoke({
        "messages": [HumanMessage(content=test_input)]
    })
    
    # Analizar qué hizo
    messages = result["messages"]
    for msg in messages:
        if hasattr(msg, 'tool_calls') and msg.tool_calls:
            for tc in msg.tool_calls:
                print(f"  🔧 Usó herramienta: {tc['name']}")
    
    final_response = messages[-1].content
    print(f"\n💬 Respuesta: {final_response}")
    print()

# 💬 SCRIPT:
print("\n💬 EXPLICAR:")
print("   'Vean cómo el agente DECIDE qué hacer basándose en el input.'")
print("   'Caso 1: Usó calculator. Caso 2: Usó search. Caso 3: Respondió directo.'")
print("   'Esto es FLEXIBILIDAD - pero es más lento y menos predecible.'")

🤖 CASO 1: Agente con input variado
📝 Input: ¿Cuánto es 5 + 3?

  🔧 Usó herramienta: calculator_tool

💬 Respuesta: 5 + 3 es igual a 8.

🤖 CASO 2: Agente con input variado
📝 Input: ¿Qué es LangGraph?

  🔧 Usó herramienta: search_tool
  🔧 Usó herramienta: search_tool
  🔧 Usó herramienta: search_tool

💬 Respuesta: No encontré información específica sobre "LangGraph". Podría ser un término nuevo o específico que no está ampliamente documentado. Si tienes más contexto o detalles sobre LangGraph, podría ayudarme a buscar información más relevante.

🤖 CASO 3: Agente con input variado
📝 Input: Hola, ¿cómo estás?


💬 Respuesta: ¡Hola! Estoy aquí y listo para ayudarte. ¿En qué puedo asistirte hoy?


💬 EXPLICAR:
   'Vean cómo el agente DECIDE qué hacer basándose en el input.'
   'Caso 1: Usó calculator. Caso 2: Usó search. Caso 3: Respondió directo.'
   'Esto es FLEXIBILIDAD - pero es más lento y menos predecible.'


## 📊 Comparación Lado a Lado

### 💬 SCRIPT:

> "Ahora que vieron ambos, comparemos directamente."

| Aspecto | Workflow | Agente |
|---------|----------|--------|
| **Flujo** | Fijo (add_edge) | Dinámico (conditional_edges) |
| **Decisiones** | Programador decide | LLM decide |
| **Velocidad** | ⚡ Rápido | 🐢 Más lento (LLM piensa) |
| **Costo** | 💰 Bajo | 💰💰 Más alto |
| **Predecible** | ✅ Siempre igual | ⚠️ Puede variar |
| **Flexible** | ❌ Inflexible | ✅ Muy flexible |

### ¿Cuándo usar cada uno?

**Usa WORKFLOW cuando:**
- ✅ Los pasos son siempre los mismos
- ✅ Performance es crítico
- ✅ Costos deben ser mínimos
- ✅ Ejemplo: Pipeline ETL, procesamiento batch

**Usa AGENTE cuando:**
- ✅ Hay incertidumbre en los pasos
- ✅ Necesitas decisiones inteligentes
- ✅ Flexibilidad > velocidad
- ✅ Ejemplo: Customer support, investigación, análisis

In [14]:
# ╔════════════════════════════════════════════════════════════╗
# ║  🧪 CELL 4: EXPERIMENTACIÓN INTERACTIVA                   ║
# ║  ⏱️ Tiempo: 3 minutos                                      ║
# ╚════════════════════════════════════════════════════════════╝

# 💬 SCRIPT: "Ahora ustedes. Modifiquen este input y vean qué pasa."

# ⬇️ MODIFICAR AQUÍ durante la demo
user_input = "¿Cuál es la capital de Francia?"

print("🧪 EXPERIMENTO: Tu turno de modificar el input\n")
print(f"Input: {user_input}\n")

result = agent_app.invoke({"messages": [HumanMessage(content=user_input)]})

print("Resultado:")
for msg in result["messages"]:
    if hasattr(msg, 'tool_calls') and msg.tool_calls:
        print(f"  🔧 Herramienta usada: {msg.tool_calls[0]['name']}")

print(f"\n💬 Respuesta: {result['messages'][-1].content}")

print("\n❓ PREGUNTA PARA LA AUDIENCIA:")
print("   '¿Qué herramienta esperan que use el agente?'")
print("   '¿Por qué eligió esa y no otra?'")

🧪 EXPERIMENTO: Tu turno de modificar el input

Input: ¿Cuál es la capital de Francia?

Resultado:

💬 Respuesta: La capital de Francia es París.

❓ PREGUNTA PARA LA AUDIENCIA:
   '¿Qué herramienta esperan que use el agente?'
   '¿Por qué eligió esa y no otra?'


## ✅ CHECKPOINT DEL INSTRUCTOR

### Antes de continuar, verificar:

- [ ] Todos vieron la diferencia en ejecución
- [ ] Todos entienden `add_edge` vs `add_conditional_edges`
- [ ] Todos saben cuándo usar workflow vs agente

### ❓ PREGUNTAS PARA HACER:

1. **"¿Cuál es la diferencia CLAVE entre workflow y agente?"**
   - Respuesta esperada: "Workflow tiene flujo fijo, agente decide dinámicamente"

2. **"Si tuvieran que procesar 1000 documentos con los mismos 3 pasos, ¿cuál usarían?"**
   - Respuesta: "Workflow - más rápido, más barato, pasos fijos"

3. **"¿Cuántos pudieron ejecutar el código en su máquina?"**
   - Contar manos - si <70%, dar tiempo extra

### 🎯 Si <70% entienden:
- Re-ejecutar ambos ejemplos con más prints
- Mostrar el estado en cada paso
- Dar 5 minutos para que experimenten

### ✅ Si ≥70% entienden:
- Continuar al siguiente notebook
- Mencionar que verán más detalles en ejercicios

## 🚀 CONTENIDO BONUS (Si hay tiempo)

### Híbridos: Lo Mejor de Ambos Mundos

En sistemas reales, a menudo combinas workflow + agente:

```python
# Workflow principal con agentes en algunos pasos
workflow:
  ├─ preprocess (workflow - siempre igual)
  ├─ analyze (AGENTE - decide qué analizar)
  ├─ validate (workflow - checks fijos)
  └─ generate_report (workflow - formato fijo)
```

💡 **Principio**: Usa workflow donde puedas, agente donde debas.

---

## 📝 Resumen

### Conceptos Clave Cubiertos:
- ✅ Workflow = flujo fijo con `add_edge`
- ✅ Agente = flujo dinámico con `add_conditional_edges`
- ✅ Trade-offs: velocidad/costo vs flexibilidad
- ✅ Cuándo usar cada uno

### 🎯 Próximo Paso:
Notebook: `02_instructor_ejercicio_1_1.ipynb` - Implementar workflow completo

---

**⏱️ Tiempo total**: ~15 minutos  
**🎓 Nivel de comprensión esperado**: 80%+
